In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', str)
from tqdm.auto import tqdm
from simulator import recommendation_media, recommendation_execution
from cvxpy_simulator import recommendation_media_cvxpy, recommendation_execution_cvxpy, binary_search_cvxpy
from calculator import scenario_execution, scenario_media

In [3]:
country = "AUS"
brand = "Smiths"

In [4]:
media_records = pd.read_excel("./db/Aus.xlsx", sheet_name="media_records_new", engine="openpyxl").drop(columns=['id', ])
execution_records = pd.read_excel("./db/Aus.xlsx", sheet_name="distribution_records_new", engine="openpyxl")
scenario_media_details = pd.read_csv("./db/scenario_media_details.csv", encoding='latin').drop(columns=['id', ])
scenario_execution_details = pd.read_csv("./db/scenario_execution_details.csv", encoding='latin').drop(columns=['id', ])

In [5]:
media_records=media_records.query("genre_platform not in ['Twitter', 'Others','Verizon']")

In [6]:
media_records_headers = media_records.columns.tolist()
execution_records_headers = execution_records.columns.tolist()
scenario_media_headers = list(set(scenario_media_details.columns.tolist()) - set(media_records_headers))
scenario_execution_headers = list(set(scenario_execution_details.columns.tolist()) - set(execution_records_headers))

In [7]:
df_constraints_media = media_records.query("country == @country and brand == @brand").replace('',np.nan).convert_dtypes()
df_constraints_execution=execution_records.loc[(execution_records['country']=='Australia')&(execution_records['current_volume_time']=='FY 2021')&(execution_records['brand'].str.startswith('Smith'))].replace('',np.nan).convert_dtypes()
df_constraints_media = df_constraints_media.reindex(columns = df_constraints_media.columns.tolist() + scenario_media_headers)
df_constraints_execution = df_constraints_execution.reindex(columns = df_constraints_execution.columns.tolist() + scenario_execution_headers)

In [8]:
#df_constraints_media = media_records.query("country == @country and brand == @brand").replace('',np.nan).convert_dtypes()
#df_constraints_execution = execution_records.query("country == @country and brand == @brand").replace('',np.nan).convert_dtypes()
#df_constraints_media = df_constraints_media.reindex(columns = df_constraints_media.columns.tolist() + scenario_media_headers)
#df_constraints_execution = df_constraints_execution.reindex(columns = df_constraints_execution.columns.tolist() + scenario_execution_headers)

In [9]:
#current_volume = df_constraints_execution.query("pack_name == 'Total'").sum().sum()
#print(f"{int(current_volume):,}")
current_volume = df_constraints_execution['current_volume'].sum()
print(current_volume)
#print(f"{int(current_volume):,}")


15971016.958844904


In [10]:
df_constraints_media = df_constraints_media[df_constraints_media["media_type"] == "Digital"].copy()

In [11]:
# new_row = df_constraints_media.loc[10, :].copy()
# new_row["media_type"] = "TV"
# new_row["input_spends"] = 0
# df_constraints_media = df_constraints_media.append(new_row,ignore_index = True)

In [12]:
# df_total = df_constraints_media[df_constraints_media['genre_platform'] == 'Total'].reset_index(drop=True)
# total_tv = df_total.query("media_type == 'TV'")

In [13]:
# df_total

In [14]:
# total_tv

In [40]:
growth_ambition_perc = 0.079
cvxpy_flag = True

if not cvxpy_flag:
    recommendation_media_df, media_convergence_flag = recommendation_media(df_constraints_media, current_volume, growth_ambition_perc)
    print(media_convergence_flag)

if cvxpy_flag:
    recommendation_media_df, media_convergence_flag = recommendation_media_cvxpy(df_constraints_media, current_volume, growth_ambition_perc)
    print(media_convergence_flag)


optimality status with SCIPY : optimal
optimal value with SCIPY : 5062425.440062818
Optimal solution found with solver SCIPY
True


C:\Users\aman.kurapa\Anaconda3\envs\kali\lib\site-packages\cvxpy\reductions\solvers\conic_solvers\scipy_conif.py:154: UserWarning: It is best to specify the 'method' parameter within scipy_options. The main advantage of this solver, is its ability to use the HiGHS LP solvers via scipy.optimize.linprog() which require a SciPy version >= 1.6.1 .

The default method 'highs' will be used in this case.

  warnings.warn("It is best to specify the 'method' parameter "


In [41]:
np.dot(recommendation_media_df.loc[0:2, "matrix_row_3_vg"], recommendation_media_df.loc[0:2, "recommendation_metric_value"])

5062425.440062818

In [42]:
np.dot(recommendation_media_df.loc[0:2, "matrix_row_2_digital"], recommendation_media_df.loc[0:2, "recommendation_metric_value"])

6690521.0

In [43]:
recommendation_media_df.loc[0:2, "recommendation_spends"].sum()

6690521.0

In [44]:
recommendation_media_df.set_index(['media_type', 'genre_platform'])['recommendation_spends'][-2:]

media_type  genre_platform
Digital     Facebook         2341682.3500000006
            Total                     6690521.0
Name: recommendation_spends, dtype: float64

In [45]:
recommendation_media_df.set_index(['media_type', 'genre_platform'])['recommendation_roi'][-2:]

media_type  genre_platform
Digital     Facebook          4.893091496743493
            Total            11.287286208334162
Name: recommendation_roi, dtype: float64

In [46]:
current_volumes=recommendation_media_df.query("media_type =='Digital' and genre_platform == 'Total' ")['current_volume'].values[0]
reco_volumes=recommendation_media_df.query("media_type =='Digital' and genre_platform == 'Total' ")['recommendation_volume'].values[0]
volume_growth=(reco_volumes-current_volumes)/current_volume
print(f"{round(volume_growth*100, 2)}%")

23.83%


In [26]:
recommendation_media_df[["lower_bounds", "recommendation_metric_value", "upper_bounds"]]

,lower_bounds,recommendation_metric_value,upper_bounds
0,7048574.449987228,84582893.39984672,84582893.39984672
1,14660750.933105959,14660750.933105959,175929011.19727147
2,43055177.62502398,301386243.3751679,516662131.50028765
3,-1.0,400629887.7081206,-1.0


In [27]:
recommendation_media_df

,country,year,timeline,analysis_period,category,brand,media_type,genre_platform,metric_type,buy_type,currency,spends_divisor,one_unit_metric_quantity,volume_unit,current_spends,current_metric_value,current_effectiveness_per_unit,current_impressions,current_volume,current_price_per_volume,current_revenue,current_roi,current_cost_per_unit,ideal_operating_point,input_cost_per_unit,input_weekly_grp,input_woa,input_spends,scenario_name,scenario_roi_output,scenario_spends,scenerio_metric_value_output,scenario_volume_output,recommendation_volume,recommendation_cost_per_unit,scenario_spends_output,recommendation_revenue,recommendation_roi,recommendation_metric_value,scenario_metric_value,recommendation_spends,scenario_revenue_output,simulation_id,scenario_cost_per_unit,current_effectiveness_per_metric,input_cost_per_metric,lower_bounds,upper_bounds,matrix_row_1_tv,matrix_row_2_digital,matrix_row_3_vg
0,AUS,2021.0,FY 21,Jan 2018 to May 2021,Snacks,Smiths,Digital,Display - Scrollx/High Impact,Impressions,fixed_spend,AUD,1000.0,1000.0,KG,233361.49366945613,4917003.808490802,49.891691856859914,-1,245317.63887222973,14.917321016175404,3659481.9699872406,15.681601589209476,47.460100247732534,-1,47.460100247732534,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,4219983.653866783,-1.0,-1.0,62950850.84774363,15.68160158920948,84582893.39984672,-1.0,4014312.5999999996,-1.0,-1.0,-1.0,0.04989169185685991,0.047460100247732534,7048574.449987228,84582893.39984672,0.0,0.047460100247732534,0.04989169185685991
1,AUS,2021.0,FY 21,Jan 2018 to May 2021,Snacks,Smiths,Digital,Dv361,Impressions,fixed_spend,AUD,1000.0,1000.0,KG,2239802.562496159,98160330.13897924,5.070475485526193,-1,497719.5476208521,14.917321016175404,7424642.267885852,3.314864618965085,22.81779777355026,-1,22.81779777355026,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,74336.97820571902,-1.0,-1.0,1108908.5672671453,3.3148646189650854,14660750.933105959,-1.0,334526.05000000005,-1.0,-1.0,-1.0,0.005070475485526193,0.02281779777355026,14660750.933105959,175929011.19727147,0.0,0.02281779777355026,0.005070475485526193
2,AUS,2021.0,FY 21,Jan 2018 to May 2021,Snacks,Smiths,Digital,Facebook,Impressions,fixed_spend,AUD,1000.0,1000.0,KG,1567254.4640884865,201713496.85604918,2.548572885704587,-1,514081.5487679844,14.917321016175404,7668719.491464654,4.8930914967434935,7.769705490787967,-1,7.769705490787967,-1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,768104.8079903167,-1.0,-1.0,11458065.994859323,4.893091496743493,301386243.3751679,-1.0,2341682.3500000006,-1.0,-1.0,-1.0,0.002548572885704587,0.007769705490787967,43055177.62502398,516662131.50028765,0.0,0.007769705490787967,0.002548572885704587
3,AUS,2021.0,FY 21,Jan 2018 to May 2021,Snacks,Smiths,Digital,Total,Impressions,fixed_spend,AUD,1000.0,1000.0,KG,4136458.728804723,307324828.9641534,-1.0,-1,1257118.7352610664,14.917321016175404,18752843.729337744,4.5335503044548915,13.45956570689966,-1,13.45956570689966,-1,-1,6690521.0,-1.0,-1.0,-1.0,-1.0,-1.0,5062425.440062818,-1.0,-1.0,75517825.40987009,11.287286208334162,400629887.7081206,-1.0,6690521.0,-1.0,-1.0,-1.0,-1.0,0.01345956570689966,-1.0,-1.0,-1.0,-1.0,-1.0


In [28]:
current_volumes=recommendation_media_df.query("media_type =='TV' and genre_platform == 'Total' ")['current_volume'].values[0]
reco_volumes=recommendation_media_df.query("media_type =='TV' and genre_platform == 'Total' ")['recommendation_volume'].values[0]
volume_growth=(reco_volumes-current_volumes)/current_volume
print(f"{round(volume_growth*100, 2)}%")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [29]:
# print(current_volume*growth_ambition_perc + recommendation_media_df.query("genre_platform == 'Total'")['current_volume'].sum())
# print(recommendation_media_df.query("genre_platform == 'Total'")['recommendation_volume'].sum())

In [30]:
# recommendation_media_df.to_csv(f"./test_output/temp/{country}/Aus_snacks_{brand}_3.csv", index=False)

# Scenario

Media

In [20]:
recommendation_media_df = pd.read_csv(f"./test_output/temp/{country}/Aus_snacks_{brand}_3.csv").replace(np.nan, -1)

In [21]:
recommendation_media_df['scenario_spends_digital_input'] = recommendation_media_df['scenario_spends']
recommendation_media_df['scenario_metric_value_input'] = recommendation_media_df['scenario_metric_value']

In [22]:
scenario_media_df = scenario_media(recommendation_media_df)

In [17]:
scenario_media_df.replace(-1, np.nan).to_csv(f"./test_output/temp/{country}/scenario/Aus_snacks_{brand}_S1.csv", index=False)

In [24]:
scenario_media_df.set_index(['media_type', 'genre_platform'])['scenario_roi'][-2:]

media_type  genre_platform
Digital     Facebook          4.893092
            Total             4.122062
Name: scenario_roi, dtype: float64